<a href="https://colab.research.google.com/github/zhestyatsky/MCL-WiC/blob/main/experiments/general_experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!pip install --upgrade pytorch-lightning scikit-learn transformers sentencepiece

     |████████████████████████████████| 819kB 16.1MB/s 
     |████████████████████████████████| 22.2MB 1.2MB/s 
     |████████████████████████████████| 1.8MB 48.5MB/s 
     |████████████████████████████████| 1.2MB 46.3MB/s 
     |████████████████████████████████| 829kB 55.0MB/s 
     |████████████████████████████████| 276kB 59.3MB/s 
     |████████████████████████████████| 102kB 14.4MB/s 
     |████████████████████████████████| 3.2MB 46.2MB/s 
     |████████████████████████████████| 890kB 45.2MB/s 
     |████████████████████████████████| 1.3MB 48.1MB/s 
     |████████████████████████████████| 296kB 52.8MB/s 
     |████████████████████████████████| 143kB 62.6MB/s 
  Created wheel for future: filename=future-0.18.2-cp36-none-any.whl size=491057 sha256=5164d069263425c9ba1e5915f2f2de319c6cf00ea110e17b10125256af5d794c
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp36-cp36m-linux_x

In [3]:
!git clone https://github.com/zhestyatsky/MCL-WiC.git MCL-WiC

import sys
sys.path.insert(0, '/content/MCL-WiC')

Cloning into 'MCL-WiC'...
remote: Enumerating objects: 422, done.
remote: Counting objects: 100% (422/422), done.
remote: Compressing objects: 100% (270/270), done.
remote: Total 422 (delta 195), reused 334 (delta 123), pack-reused 0
Receiving objects: 100% (422/422), 3.13 MiB | 19.40 MiB/s, done.
Resolving deltas: 100% (195/195), done.


In [4]:
### THIS IS WHERE YOU SELECT THE MODEL ###
### ALL AVAILABLE MODELS ARE LISTED IN src/training/config.py ###

MODEL_DESCRIPTION = "bert-large-cos_sim-relu"

In [5]:
from src.data.dataset import get_train_val_test_loaders
from src.training.models import get_model
from src.training.trainer import get_trainer
from src.util.util import seed_everything

seed_everything(42)

train_loader, val_loader, test_loader = get_train_val_test_loaders(MODEL_DESCRIPTION)
clf = get_model(MODEL_DESCRIPTION)
trainer = get_trainer()

trainer.fit(clf, train_loader, val_loader)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores

  | Name           | Type             | Params
----------------------------------------------------
0 | model          | BertModel        | 333 M 
1 | loss           | BCELoss          | 0     
2 | valid_accuracy | Accuracy         | 0     
3 | test_accuracy  | Accuracy         | 0     
4 | activation     | ReLU             | 0     
5 | cos            | CosineSimilarity | 0     
----------------------------------------------------
333 M     Trainable params
0         Non-trainable params
333 M     Total params
1,334.317 Total estimated model params size (MB)


/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)


1

In [6]:
trainer.test(clf, val_loader)


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{}
--------------------------------------------------------------------------------


[{}]

In [7]:
from src.training.hyperparams import find_threshold_if_needed

threshold, validation_accuracy = find_threshold_if_needed(val_loader.dataset.labels, clf.predictions_proba, MODEL_DESCRIPTION)
trainer.test(clf, test_loader)

Threshold: 0.6434575915336609, Validation Accuracy: 0.7098445892333984



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{}
--------------------------------------------------------------------------------


[{}]

In [8]:
from src.testing.testing import get_results, write_outputs

results = get_results(clf.predictions_proba, threshold, test_loader.dataset)
outputs = write_outputs(results, MODEL_DESCRIPTION, '/content/gdrive/My Drive/mcl-wic-outputs/', validation_accuracy, threshold)